In [2]:
# importer les packages
import pandas as pd
from requests import get
from bs4 import BeautifulSoup as bs

In [3]:
# url
url = 'https://dakar-auto.com/senegal/voitures-4?&page=1'
res = get(url)
soup = bs(res.text, 'html.parser')
containers =soup.find_all( 'div', class_ = 'listings-cards__list-item mb-md-3 mb-3')
len(containers)

20

In [4]:
# premier container
container = containers[0]
container

<div class="listings-cards__list-item mb-md-3 mb-3">
<div class="listing-card">
<div class="listing-card__aside">
<a class="listing-card__aside__inner d-block" href="/senegal/voitures/occasion/coupes-and-cabriolets/citroen/c3/annonce-139356" title="Citroen C3 2016">
<img alt="Citroen C3 2016" class="img-fluid" height="194" loading="lazy" src="/img/listings/2024/09/IMG-20240927-WA0050_300x194.jpg" width="300"/> <div class="listing-card__images">
<span>
<i class="fas fa-camera"></i>
<span class="listing-card__images-count px-1">2</span>
</span>
</div>
</a>
</div>
<div class="listing-card__content p-2">
<div class="listing-card__content__inner">
<div class="listing-card__header">
<div class="listing-card__header__content">
<h4 class="listing-card__header__category font-weight-lighter mb-1">
		Coupés &amp; cabriolets					<small class="badge badge-pill badge-primary ml-2">Occasion</small>
</h4>
<h2 class="listing-card__header__title mb-md-2 mb-0" style="white-space: normal;">
<a href="/sene

In [ ]:
marque_annee = container.find('h2', class_= 'listing-card__header__title mb-md-2 mb-0').find('a').text.strip()
#marque_annee
prix = container.find('h3',class_= 'listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace('F CFA','').replace('\u202f',' ').strip()
#prix
kilometrage = container.find('ul', class_ = 'list-inline').find_all('li')[1].text.replace('km','').strip()
#kilometrage = container.select_one('ul li.listing-card__attribute.list-inline-item:nth-child(2)').text.replace("km", "").strip()
#kilometrage
#boite_vitesse = container.select_one('ul li.listing-card__attribute.list-inline-item:nth-child(3)').text.strip()
boite_vitesse = container.find('ul', class_ = 'list-inline').find_all('li')[2].text.strip()
#boite_vitesse
carburant = container.find('ul', class_ = 'list-inline').find_all('li')[3].text.strip()
#carburant
proprietaire = container.find('p', class_ = 'time-author m-0').find('a').text
#proprietaire



' Par Bintou Sylla'

In [26]:
# scraper les données de la première page
data = []
for container in containers:
        try:
            marque_annee = container.find('h2', class_= 'listing-card__header__title mb-md-2 mb-0').find('a').text.strip()
            prix = container.find('h3',class_= 'listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace('F CFA','').replace('\u202f',' ').strip()
            kilometrage = container.find('ul', class_ = 'list-inline').find_all('li')[1].text.replace('km','').strip()
            boite_vitesse = container.find('ul', class_ = 'list-inline').find_all('li')[2].text.strip()
            carburant = container.find('ul', class_ = 'list-inline').find_all('li')[3].text.strip()
            proprietaire = container.find('p', class_ = 'time-author m-0').find('a').text
            dic = {'marque_annee': marque_annee, 'prix': prix, 'kilometrage': kilometrage, 'boite_vitesse':boite_vitesse, 'carburant':carburant,'proprietaire':proprietaire}
            data.append(dic)
        except:
            pass

DF = pd.DataFrame(data)

In [27]:
DF.shape

(19, 6)

In [29]:
DF.head()

,marque_annee,prix,kilometrage,boite_vitesse,carburant,proprietaire
0,Citroen C3 2016,5 000 000,150000,Manuelle,Diesel,Par Bintou Sylla
1,Mitsubishi Outlander 2016,9 500 000,115000,Automatique,Essence,Par Sokhna Faty
2,Renault megane-3 2013,4 000 000,260000,Manuelle,Diesel,Par Gm DK
3,Rover range 2017,15 499 000,54000,Automatique,Essence,Par Chasseur Automobile
4,Ford Fusion 2013,3 950 000,80000,Automatique,Essence,Par Fatou Kiné


In [30]:
DF.isna().sum()

marque_annee     0
prix             0
kilometrage      0
boite_vitesse    0
carburant        0
proprietaire     0
dtype: int64

In [31]:
# scraper les données sur toutes les pages
df = pd.DataFrame()
for p in range(1,5):
    url = f'https://dakar-auto.com/senegal/voitures-4?&page={p}'
    res = get(url)
    soup = bs(res.text, 'html.parser')
    containers =soup.find_all( 'div', class_ = 'listings-cards__list-item mb-md-3 mb-3')
    data = []
    for container in containers:
        try:
            marque_annee = container.find('h2', class_= 'listing-card__header__title mb-md-2 mb-0').find('a').text.strip()
            prix = container.find('h3',class_= 'listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace('F CFA','').replace('\u202f',' ').strip()
            kilometrage = container.find('ul', class_ = 'list-inline').find_all('li')[1].text.replace('km','').strip()
            boite_vitesse = container.find('ul', class_ = 'list-inline').find_all('li')[2].text.strip()
            carburant = container.find('ul', class_ = 'list-inline').find_all('li')[3].text.strip()
            proprietaire = container.find('p', class_ = 'time-author m-0').find('a').text
            dic = {'marque_annee': marque_annee, 'prix': prix, 'kilometrage': kilometrage, 'boite_vitesse':boite_vitesse, 'carburant':carburant,'proprietaire':proprietaire}
            data.append(dic)
        except:
            pass

    DF = pd.DataFrame(data)
    df = pd.concat([df,DF], axis = 0).reset_index(drop = True)

In [32]:
df.shape

(78, 6)

In [34]:
df.head()['prix']

0     5 000 000
1     9 500 000
2     4 000 000
3    15 499 000
4     3 950 000
Name: prix, dtype: object

# Fonction pour scraper les donnees auto avec bs4

In [35]:
def bs4_auto_data_scrape(pages):
    df = pd.DataFrame()
    for p in range(1, pages + 1):
        
        url = f'https://dakar-auto.com/senegal/voitures-4?&page={p}'
        res = get(url)
        soup = bs(res.text, 'html.parser')
        containers =soup.find_all( 'div', class_ = 'listings-cards__list-item mb-md-3 mb-3')
        data = []
        for container in containers:
            try:
                marque_annee = container.find('h2', class_= 'listing-card__header__title mb-md-2 mb-0').find('a').text.strip()
                prix = container.find('h3',class_= 'listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace('F CFA','').replace('\u202f',' ').strip()
                kilometrage = container.find('ul', class_ = 'list-inline').find_all('li')[1].text.replace('km','').strip()
                boite_vitesse = container.find('ul', class_ = 'list-inline').find_all('li')[2].text.strip()
                carburant = container.find('ul', class_ = 'list-inline').find_all('li')[3].text.strip()
                proprietaire = container.find('p', class_ = 'time-author m-0').find('a').text
                dic = {'marque_annee': marque_annee, 'prix': prix, 'kilometrage': kilometrage, 'boite_vitesse':boite_vitesse, 'carburant':carburant,'proprietaire':proprietaire}
                data.append(dic)
            except:
                pass
        
        DF = pd.DataFrame(data)
        df = pd.concat([df, DF], axis=0).reset_index(drop=True)
    
    return df


In [38]:
df=bs4_auto_data_scrape(10)

In [39]:
df.shape

(194, 6)

In [43]:
df.head(25)

,marque_annee,prix,kilometrage,boite_vitesse,carburant,proprietaire
0,Citroen C3 2016,5 000 000,150000,Manuelle,Diesel,Par Bintou Sylla
1,Mitsubishi Outlander 2016,9 500 000,115000,Automatique,Essence,Par Sokhna Faty
2,Renault megane-3 2013,4 000 000,260000,Manuelle,Diesel,Par Gm DK
3,Rover range 2017,15 499 000,54000,Automatique,Essence,Par Chasseur Automobile
4,Ford Fusion 2013,3 950 000,80000,Automatique,Essence,Par Fatou Kiné
5,Hyundai Santa Fe 2020,5 000 000,86500,Automatique,Essence,Par MOURAD ENNAJY
6,Hyundai TUCSON--LIMITED 2016,4 000 000,88000,Automatique,Essence,Par MOURAD ENNAJY
7,Toyota Fortuner 2014,19 500 000,20500,Manuelle,Diesel,Par Cheikh Ahmed Tidiane SY
8,Nissan ROGUE 2016,8 500 000,138000,Automatique,Essence,Par Makhou Diagne
9,Dacia Sandero 2018,5 500 000,60000,Manuelle,Essence,Par Daouda Fall


# Fonction   pour scraper les donnees scooters avec bs4

In [87]:
def _bs4_scooters_data_scrape(pages):
    df = pd.DataFrame()
    for p in range(1, pages + 1):
        
        url = f'https://dakar-auto.com/senegal/motos-and-scooters-3?&page={p}'
        res = get(url)
        soup = bs(res.text, 'html.parser')
        containers =soup.find_all( 'div', class_ = 'listings-cards__list-item mb-md-3 mb-3')
        data = []
        for container in containers:
            try:
                marque_annee = container.find('h2', class_= 'listing-card__header__title mb-md-2 mb-0').find('a').text.strip()
                prix = container.find('h3',class_= 'listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace('F CFA','').replace('\u202f',' ').strip()
                adresse=container.find('div',class_='col-12 entry-zone-address').text.replace("\n","")
                kilometrage = container.find('ul', class_ = 'list-inline').find_all('li')[1].text.replace('km','').strip()
                proprietaire = container.find('p', class_ = 'time-author m-0').find('a').text
                dic = {'marque_annee': marque_annee, 'prix': prix, 'adresse': adresse,'kilometrage':kilometrage,'proprietaire':proprietaire}
                data.append(dic)
            except:
                pass
        
        DF = pd.DataFrame(data)
        df = pd.concat([df, DF], axis=0).reset_index(drop=True)
    
    return df
        


In [89]:
scooters=_bs4_scooters_data_scrape(1)

In [90]:
scooters.shape

(20, 5)

In [95]:
scooters.head()

,marque_annee,prix,adresse,kilometrage,proprietaire
0,Yamaha TMax 2020,3 900 000,"Ouakam,Dakar",14000,Par Cheikh Mbacké
1,Yamaha X-Max 2020,1 200 000,"Yoff,Dakar",10000,Par Mohamed SALL
2,Honda CBF 2014,1 150 000,"Grand-Dakar,Dakar",55000,Par Mohamed Memed
3,Piaggio Xevo 2012,490 000,"Parcelles Assainies,Dakar",1,Par Djibril Ba
4,Aprilia Mojito 2018,300 000,"Grand-Dakar,Dakar",150,Par Ibrahim Sarr


# Fonction   pour scraper les donnees de voiture de location avec bs4

In [92]:
def bs4_rented_auto_data_scrape(pages):
    
    df = pd.DataFrame()
    for p in range(1, pages + 1):
        
        url = f'https://dakar-auto.com/senegal/location-de-voitures-19?&page={p}'
        res = get(url)
        soup = bs(res.text, 'html.parser')
        containers =soup.find_all( 'div', class_ = 'listings-cards__list-item mb-md-3 mb-3')
        data = []
        for container in containers:
            try:
                marque_annee = container.find('h2', class_= 'listing-card__header__title mb-md-2 mb-0').find('a').text.strip()
                prix = container.find('h3',class_= 'listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace('F CFA','').replace('\u202f',' ').strip()
                adresse=container.find('div',class_='col-12 entry-zone-address').text.replace("\n","")
                proprietaire = container.find('p', class_ = 'time-author m-0').find('a').text
                dic = {'marque_annee': marque_annee, 'prix': prix, 'adresse': adresse,'proprietaire':proprietaire}
                data.append(dic)
            except:
                pass
        
        DF = pd.DataFrame(data)
        df = pd.concat([df, DF], axis=0).reset_index(drop=True)
    
    return df
               

In [93]:
rented_auto=bs4_rented_auto_data_scrape(1)

In [94]:
rented_auto.head()

,marque_annee,prix,adresse,proprietaire
0,Citroen Berlingo 2014,20 000,"Médina,Dakar",Par Chasseur Automobile
1,Peugeot 308 2013,30 000,"Grand-Yoff,Dakar",Par Chic Drive
2,Ford Fiesta 2014,30 000,"Sangalkam,Dakar",Par Medoune Ndiaye
3,Ford Fusion 2015,35 000,"Cité Keur gorgui,Dakar",Par Pape Cheikh automobile
4,Toyota Rav 4 2020,35 000,"Mermoz Sacré-Coeur,Dakar",Par LocAuto Sénégal Location de voiture


In [62]:

url = f'https://dakar-auto.com/senegal/location-de-voitures-19?&page=1'
res = get(url)
soup = bs(res.text, 'html.parser')
containers =soup.find_all( 'div', class_ = 'listings-cards__list-item mb-md-3 mb-3')
len(containers)

20

In [63]:
containers

[<div class="listings-cards__list-item mb-md-3 mb-3">
 <div class="listing-card">
 <div class="listing-card__aside">
 <a class="listing-card__aside__inner d-block" href="/senegal/location-de-voitures/occasion/location-de-voitures-1/citroen/berlingo/annonce-138611" title="Citroen Berlingo 2014">
 <img alt="Citroen Berlingo 2014" class="img-fluid" height="194" loading="lazy" src="/img/listings/2024/04/44e3ee82-d6e9-4e42-8ea3-d7642bcf6090_300x194.jpg" width="300"/> <div class="listing-card__images">
 <span>
 <i class="fas fa-camera"></i>
 <span class="listing-card__images-count px-1">2</span>
 </span>
 </div>
 </a>
 </div>
 <div class="listing-card__content p-2">
 <div class="listing-card__content__inner">
 <div class="listing-card__header">
 <div class="listing-card__header__content">
 <h4 class="listing-card__header__category font-weight-lighter mb-1">
 		Location de voitures					<small class="badge badge-pill badge-primary ml-2">Occasion</small>
 </h4>
 <h2 class="listing-card__header_

In [64]:
# premier container
container = containers[0]
container

<div class="listings-cards__list-item mb-md-3 mb-3">
<div class="listing-card">
<div class="listing-card__aside">
<a class="listing-card__aside__inner d-block" href="/senegal/location-de-voitures/occasion/location-de-voitures-1/citroen/berlingo/annonce-138611" title="Citroen Berlingo 2014">
<img alt="Citroen Berlingo 2014" class="img-fluid" height="194" loading="lazy" src="/img/listings/2024/04/44e3ee82-d6e9-4e42-8ea3-d7642bcf6090_300x194.jpg" width="300"/> <div class="listing-card__images">
<span>
<i class="fas fa-camera"></i>
<span class="listing-card__images-count px-1">2</span>
</span>
</div>
</a>
</div>
<div class="listing-card__content p-2">
<div class="listing-card__content__inner">
<div class="listing-card__header">
<div class="listing-card__header__content">
<h4 class="listing-card__header__category font-weight-lighter mb-1">
		Location de voitures					<small class="badge badge-pill badge-primary ml-2">Occasion</small>
</h4>
<h2 class="listing-card__header__title mb-md-2 mb-0"

In [65]:
marque_annee = container.find('h2', class_= 'listing-card__header__title mb-md-2 mb-0').find('a').text.strip()
marque_annee

'Citroen Berlingo 2014'

In [66]:
prix = container.find('h3',class_= 'listing-card__header__price font-weight-bold text-uppercase mb-0').text.strip().replace('F CFA','').replace('\u202f',' ').strip()
prix

'20 000'

In [77]:
adresse=container.find('div',class_='col-12 entry-zone-address').text.replace("\n","")
adresse

'Médina,Dakar'

In [71]:
proprietaire = container.find('p', class_ = 'time-author m-0').find('a').text
proprietaire

' Par Chasseur  Automobile'

In [80]:
rented_auto=bs4_rented_auto_data_scrape(1)

In [81]:
rented_auto.shape

(20, 4)

In [82]:
rented_auto.head()

,marque_annee,prix,adresse,proprietaire
0,Citroen Berlingo 2014,20 000,"Médina,Dakar",Par Chasseur Automobile
1,Peugeot 308 2013,30 000,"Grand-Yoff,Dakar",Par Chic Drive
2,Ford Fiesta 2014,30 000,"Sangalkam,Dakar",Par Medoune Ndiaye
3,Ford Fusion 2015,35 000,"Cité Keur gorgui,Dakar",Par Pape Cheikh automobile
4,Toyota Rav 4 2020,35 000,"Mermoz Sacré-Coeur,Dakar",Par LocAuto Sénégal Location de voiture


In [ ]:
# premier container
container = containers[0]
url_container = container.find('a', class_ = 'listing-card__aside__inner d-block')['href']
url_container
#res_c = get(url_container)
#soup_c = bs(res_c.text, 'html.parser')

'/senegal/voitures/occasion/coupes-and-cabriolets/citroen/c3/annonce-139356'